# How far kicker from me ? (Long Snapper's asking)
Or how i realize that there is a Punter(Holder) among Kickers in tracking.position

import libraries and load data

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


!pip install joypy
from joypy import joyplot
import math
import seaborn as sns

In [ ]:
game = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/games.csv')
play = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/plays.csv')
tracking2018 = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/tracking2018.csv')
tracking2019 = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/tracking2019.csv')
tracking2020 = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/tracking2020.csv')

In [ ]:
tracking = tracking2018
tracking.append(tracking2019)
tracking.append(tracking2020)

print('')

# I. Field goals

LS and K from only one team on field at moment of snap . Get the coordinates of these two. 
(I assume that field position of a player in tracking-data and position of player in players-data the same or even better)

In [ ]:
data_fg = pd.merge(play[play.specialTeamsPlayType=='Field Goal'],
               tracking[(tracking.position=='LS') & (tracking.event == 'ball_snap')],
               how = 'inner', on = ['playId','gameId'])
data_fg = data_fg[['gameId','playId','x','y']]

# add kickers info
kickers = pd.merge(play[play.specialTeamsPlayType=='Field Goal'],
               tracking[(tracking.position=='K') & (tracking.event == 'ball_snap')],
               how = 'inner', on = ['playId','gameId'])
data_fg = pd.merge(data_fg, kickers, how = 'inner', on = ['playId','gameId'], 
                   suffixes=("_ls", "_k") )
data_fg = data_fg[['x_ls','y_ls', 'x_k','y_k', 'playDirection','displayName','specialTeamsResult','possessionTeam']]
#data_fg.head(3)

In [ ]:
# distance to goal post (i assume that starting point coinside with LS coordinates)
distance_to_goal = []
# distance between Kicker and Long Snapper
distance_between = []
# is kick good?
fg_goal =[]

for i in range(len(data_fg)):
    distance_between.append(math.sqrt((data_fg['x_ls'][i]-data_fg['x_k'][i])**2+(data_fg['y_ls'][i]-data_fg['y_k'][i])**2))
    if data_fg.playDirection[i]=='left':
        distance_to_goal.append(math.sqrt((data_fg['x_ls'][i])**2 + (data_fg['y_ls'][i]-53.33/2)**2))
    else:
        distance_to_goal.append(math.sqrt((120-data_fg['x_ls'][i])**2 + (data_fg['y_ls'][i]-53.33/2)**2))
    if data_fg.specialTeamsResult[i]=='Kick Attempt Good':
        fg_goal.append(1)
    else:
        fg_goal.append(0)
data_fg = pd.concat([data_fg,
                     pd.DataFrame(distance_to_goal,columns=['distance_to_goal']),
                     pd.DataFrame(distance_between,columns=['distance_between']),
                     pd.DataFrame(fg_goal,columns=['fg_goal'])], axis=1)
#data_fg.head(5)

In [ ]:
ax = sns.scatterplot(x='distance_to_goal', y='distance_between',
                hue="specialTeamsResult", #size="depth",
                #palette="ch:r=-.2,d=.3_r",
                #hue_order=clarity_ranking,
                sizes=(1, 8), linewidth=0,
                data=data_fg
               )
ax.set(title ='Scatter plot!')

Look like distance for run-up is not depend on distance to goal post? (open quation)

In [ ]:
ax = sns.displot(data=data_fg[['distance_between','specialTeamsResult']],
                 x="distance_between", binwidth=0.1, 
                 hue="specialTeamsResult", multiple="stack")
ax.set(title='Field goals \n Distance between LS and K at moment of snap (yards)')

In [ ]:
fg_visual = data_fg
# filter by number of field-goal attempts
fg_visual = fg_visual[fg_visual['displayName'].map(fg_visual['displayName'].value_counts()) >= 5]

#fg_visual['order'] = fg_visual.groupby('displayName').distance_between.transform(lambda x: np.mean(x))
fg_visual['order'] = fg_visual.groupby('displayName').fg_goal.transform(lambda x: np.average(x))

fg_visual = fg_visual.sort_values(['order'], ascending=False)

fg_grouped = fg_visual.groupby("displayName", sort=False)

joyplot(fg_grouped, column = 'distance_between', figsize = (10,10), 
        overlap = 0.5, color = 'lightblue')

plt.xticks([0.0, 2.5, 5.0, 7.5, 10.0, 12.5])
plt.rc('grid', linestyle=':', color='lightgray', linewidth=0.5)
plt.grid(True, zorder = 0)
plt.title("Density of field-goal attempts \n Ordered by field goal accuracy \n best on top", fontsize = 16)
plt.xlabel("Distance to LS", fontsize = 14)
plt.show()

Looks like distance to Long Snapper is something personal

# II. Extra points

In [ ]:
data_ep = pd.merge(play[play.specialTeamsPlayType=='Extra Point'],
               tracking[(tracking.position=='LS') & (tracking.event == 'ball_snap')],
               how = 'inner', on = ['playId','gameId'])
data_ep = data_ep[['gameId','playId','x','y']]

# add kickers info
kickers = pd.merge(play[play.specialTeamsPlayType=='Extra Point'],
               tracking[(tracking.position=='K') & (tracking.event == 'ball_snap')],
               how = 'inner', on = ['playId','gameId'])
data_ep = pd.merge(data_ep, kickers, how = 'inner', on = ['playId','gameId'], 
                   suffixes=("_ls", "_k") )
data_ep = data_ep[['x_ls','y_ls', 'x_k','y_k', 'playDirection','displayName','specialTeamsResult','possessionTeam']]

distance_between = []
# is kick good?
ep_goal =[]

for i in range(len(data_ep)):
    distance_between.append(math.sqrt((data_ep['x_ls'][i]-data_ep['x_k'][i])**2+(data_ep['y_ls'][i]-data_ep['y_k'][i])**2))
    if data_ep.playDirection[i]=='left':
        distance_to_goal.append(math.sqrt((data_ep['x_ls'][i])**2 + (data_ep['y_ls'][i]-53.33/2)**2))
    else:
        distance_to_goal.append(math.sqrt((120-data_ep['x_ls'][i])**2 + (data_ep['y_ls'][i]-53.33/2)**2))
    if data_ep.specialTeamsResult[i]=='Kick Attempt Good':
        ep_goal.append(1)
    else:
        ep_goal.append(0)
data_ep = pd.concat([data_ep,
                     pd.DataFrame(distance_between,columns=['distance_between']),
                     pd.DataFrame(ep_goal,columns=['ep_goal'])], axis=1)
#data_ep.head(5)

In [ ]:
ax = sns.displot(data=data_ep[['distance_between','specialTeamsResult']],
                 x="distance_between", binwidth=0.1, 
                 hue="specialTeamsResult", multiple="stack")
ax.set(title='Extra points \n Distance between LS and K at moment of snap (yards)')

In [ ]:
ep_visual = data_ep
# filter by number of field-goal attempts
ep_visual = ep_visual[ep_visual['displayName'].map(ep_visual['displayName'].value_counts()) >= 5]

#ep_visual['order'] = ep_visual.groupby('displayName').distance_between.transform(lambda x: np.mean(x))
ep_visual['order'] = ep_visual.groupby('displayName').ep_goal.transform(lambda x: np.average(x))

ep_visual = ep_visual.sort_values(['order'], ascending=False)

ep_grouped = ep_visual.groupby("displayName", sort=False)

joyplot(ep_grouped, column = 'distance_between', figsize = (10,10), 
        overlap = 0.5, color = 'lightblue')

plt.xticks([0.0, 2.5, 5.0, 7.5, 10.0, 12.5])
plt.rc('grid', linestyle=':', color='lightgray', linewidth=0.5)
plt.grid(True, zorder = 0)
plt.title("Density of field-goal attempts \n Ordered by field goal accuracy \n (best on top)", fontsize = 16)
plt.xlabel("Distance to LS", fontsize = 14)
plt.show()

One Kicker has original style ( except that he worst on field goals)
Look at map of his field goals attempts

# III. Map of field goals

In [ ]:
data_s = pd.merge(play[play.specialTeamsPlayType=='Field Goal'],
               tracking[(tracking.position=='LS') & (tracking.event == 'ball_snap')],
               how = 'inner', on = ['playId','gameId'])
data_s = pd.merge(data_s,game,
                   how = 'inner', on = 'gameId')
data_s = data_s[['gameId','playId','x','y','homeTeamAbbr','visitorTeamAbbr']]
# add kickers info
kickers = pd.merge(play[play.specialTeamsPlayType=='Field Goal'],
               tracking[(tracking.position=='K') & (tracking.event == 'ball_snap') 
                        & (tracking.displayName=='Matt Wile')],
               how = 'inner', on = ['playId','gameId'])
data_s = pd.merge(data_s, kickers, how = 'inner', on = ['playId','gameId'], 
                   suffixes=("_ls", "_k") )
data_s = data_s[['x_ls','y_ls', 'x_k','y_k', 'playDirection','displayName','specialTeamsResult','possessionTeam','homeTeamAbbr','visitorTeamAbbr']]
#data_fg.head(3)

In [ ]:
from matplotlib import cbook
# its local file
imageFile = cbook.get_sample_data('/kaggle/input/nfl2022/nfl_field5.png')
image = plt.imread(imageFile)
extent = -60, 60, 0, 53.3

data_s.loc[(data_s['playDirection'] == "left") & (data_s['possessionTeam']==data_s['homeTeamAbbr']), 'x_ls'] = 120-data_s.loc[(data_s['playDirection'] == "left") & (data_s['possessionTeam']==data_s['homeTeamAbbr']), 'x_ls']
data_s.loc[(data_s['playDirection'] == "right") & (data_s['possessionTeam']==data_s['visitorTeamAbbr']), 'x_ls'] = 120-data_s.loc[(data_s['playDirection'] == "right") & (data_s['possessionTeam']==data_s['visitorTeamAbbr']), 'x_ls']


data_s.loc[(data_s['playDirection'] == "left") & (data_s['possessionTeam']==data_s['homeTeamAbbr']), 'y_ls'] = 160/3-data_s.loc[(data_s['playDirection'] == "left") & (data_s['possessionTeam']==data_s['homeTeamAbbr']), 'y_ls']
data_s.loc[(data_s['playDirection'] == "right") & (data_s['possessionTeam']==data_s['visitorTeamAbbr']), 'y_ls'] = 160/3-data_s.loc[(data_s['playDirection'] == "right") & (data_s['possessionTeam']==data_s['visitorTeamAbbr']), 'y_ls']

plt.plot(data_s[['x_ls']][data_s.specialTeamsResult=='Kick Attempt Good']*1215/120,
         data_s[['y_ls']][data_s.specialTeamsResult=='Kick Attempt Good']*553/53.3, 'o', markersize=8, color='cyan', alpha=0.9)

plt.plot(data_s[['x_ls']][data_s.specialTeamsResult=='Kick Attempt No Good']*1215/120,
         data_s[['y_ls']][data_s.specialTeamsResult =='Kick Attempt No Good']*553/53.3, 'o', markersize=8, color='red', alpha=0.9)

plt.rcParams["figure.figsize"] = (20,10)
plt.axis('off')
plt.title("Matt Wile Field Goal Attempts")
plt.imshow(image)

check this (all goals where made onroad)

In [ ]:
data_s

In [ ]:
kickers

But there is no nflId of Matt Wile among kicker_Id in plays-data.
(his position , mentioned in data is 'K', but he plays on another position - Holder/Punter)